<a href="https://colab.research.google.com/github/brendanpshea/data_clean_nypl/blob/main/New_York_Public_Library_Menus_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New York Public Library-Menus-EDA

# Phase-I Report Outline

The project instructions (separate document) describe what exactly you should do as part of Phase-I of the project. The following outline should be followed for your Phase-I report:

## 1. Description of Dataset (25 points)
### a. ER Diagram, Ontology, or Database Schema (10 points)
   - Provide an **Entity-Relationship (ER) diagram**, an **ontology**, or a detailed **database schema**.
### b. Narrative Description of the Dataset (15 points)
   - Provide a **narrative description** of the dataset covering its **structure** and **content**.

## 2. Use Cases (30 points)
### a. Target (Main) Use Case U1: Data Cleaning is Necessary and Sufficient (20 points)
   - Describe a main use case where data cleaning is both necessary and sufficient.
### b. "Zero Data Cleaning" Use Case U0: Data Cleaning is Not Necessary (5 points)
   - Describe a use case where data cleaning is not necessary.
### c. "Never Enough" Use Case U2: Data Cleaning is Not Sufficient (5 points)
   - Describe a use case where data cleaning is not sufficient.

## 3. Data Quality Problems (30 points)
### a. List of Obvious Data Quality Problems with Evidence (20 points)
   - List **obvious data quality problems** with **evidence** (examples and/or screenshots).
### b. Explanation of Data Cleaning Necessity for Main Use Case U1 (10 points)
   - Explain **why** and **how** data cleaning is necessary to support the main use case U1.

## 4. Initial Plan for Phase-II (15 points)
Below is a possible plan, listing typical data cleaning workflow steps. In your Plan for Phase-II, fill in additional details for the project steps as needed. In particular, include who of your team members will be responsible for which steps, and list the timeline that you are setting yourselves!

### a. Review and Update Use Case and Dataset Descriptions
   - **S1**: Review (and update if necessary) your use case description and dataset description.

### b. Profile Dataset to Identify Data Quality Problems
   - **S2**: Profile the dataset (D) to identify data quality (DQ) problems.
     - How do you plan to do it?
     - What tools are you going to use?

### c. Perform Data Cleaning Proper
   - **S3**: Perform data cleaning (DC) proper.
     - How are you going to do it?
     - What tools do you plan to use?
     - Who does what?

### d. Data Quality Checking
   - **S4**: Check if the cleaned dataset (D’) is really “cleaner” than the original dataset (D).
     - Develop test examples / demos.

### e. Document and Quantify Change
   - **S5**: Document and quantify the changes.
     - Columns and cells changed
     - Integrity constraint (IC) violations detected: before vs. after


Brendan's Notes:

The original data files are available here:

https://uofi.app.box.com/s/zh2hxfkq0cc6vyftw91nqa4smdpq7ybk

I just downloaded all the files (as a zip), and stuck them on dropbox (see below). From there, I just ran the standard Pandas commands to get an overview of the data.


In [2]:
# Get the files from BRendan's drobbox
!wget -q -nc "https://www.dropbox.com/scl/fi/l8b5np5xoes57nr1hqhae/NYPL-menus.zip?rlkey=pak0ox3wae0x0yd09d23eoqma&st=mfq9dka9&dl=1" -O NYPL-menus.zip

In [3]:
# Unzip them. They should be in the "NYPL-menus" directory
!unzip "NYPL-menus.zip"
!ls

Archive:  NYPL-menus.zip
  inflating: NYPL-menus/Menu.csv     
  inflating: NYPL-menus/MenuItem.csv  
  inflating: NYPL-menus/Dish.csv     
  inflating: NYPL-menus/MenuPage.csv  
NYPL-menus  NYPL-menus.zip  sample_data


## ERD Diagram
Here's an initial ERD (will need to double-checked for accuracy)

In [20]:
import base64
from IPython.display import Image, display, HTML

def mm(graph):
    graphbytes = graph.encode("utf8")
    base64_bytes = base64.b64encode(graphbytes)
    base64_string = base64_bytes.decode("ascii")
    display(Image(url="https://mermaid.ink/img/" + base64_string))


mm("""
erDiagram
    DISH {
        int id PK
        string name
        float description
        int menus_appeared
        int times_appeared
        int first_appeared
        int last_appeared
        float lowest_price
        float highest_price
    }

    MENU {
        int id PK
        string name
        string sponsor
        string event
        string venue
        string place
        string physical_description
        string occasion
        string notes
        string call_number
        float keywords
        float language
        string date
        string location
        float location_type
        string currency
        string currency_symbol
        string status
        int page_count
        int dish_count
    }

    MENUITEM {
        int id PK
        int menu_page_id FK
        float price
        float high_price
        float dish_id FK
        string created_at
        string updated_at
        float xpos
        float ypos
    }

    MENUPAGE {
        int id PK
        int menu_id FK
        string page_number
        string image_id
        string full_height
        string full_width
        string uuid
        string created_at
        string updated_at
    }

    MENU ||--o{ MENUPAGE : "contains"
    MENUITEM }o--|| DISH : "refers to"
    MENUPAGE ||--o{ MENUITEM : "includes"
""")

### Explore Data Using Pandas

In [5]:
import pandas as pd
import numpy as np

dish_df =  pd.read_csv('NYPL-menus/Dish.csv')
menu_df = pd.read_csv('NYPL-menus/Menu.csv')
menuitem_df = pd.read_csv('NYPL-menus/MenuItem.csv')
menupage_df = pd.read_csv('NYPL-menus/MenuPage.csv')

In [6]:
dish_df.head()

,id,name,description,menus_appeared,times_appeared,first_appeared,last_appeared,lowest_price,highest_price
0,1,Consomme printaniere royal,NaN,8,8,1897,1927,0.20,0.4
1,2,Chicken gumbo,NaN,111,117,1895,1960,0.10,0.8
2,3,Tomato aux croutons,NaN,13,13,1893,1917,0.25,0.4
3,4,Onion au gratin,NaN,41,41,1900,1971,0.25,1.0
4,5,St. Emilion,NaN,66,68,1881,1981,0.00,18.0


In [7]:
dish_df.shape

(423397, 9)

In [8]:
dish_df.describe().round(2)

,id,description,menus_appeared,times_appeared,first_appeared,last_appeared,lowest_price,highest_price
count,423397.00,0.0,423397.00,423397.00,423397.00,423397.00,394297.00,394297.0
mean,264456.59,NaN,3.06,3.15,1675.51,1679.30,0.97,1.6
std,150489.07,NaN,27.82,29.96,651.32,651.93,6.71,12.7
min,1.00,NaN,0.00,-6.00,0.00,0.00,0.00,0.0
25%,132374.00,NaN,1.00,1.00,1900.00,1900.00,0.00,0.0
50%,269636.00,NaN,1.00,1.00,1914.00,1917.00,0.00,0.0
75%,397135.00,NaN,1.00,1.00,1949.00,1955.00,0.40,0.6
max,515677.00,NaN,7740.00,8484.00,2928.00,2928.00,1035.00,3050.0


In [9]:
dish_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423397 entries, 0 to 423396
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              423397 non-null  int64  
 1   name            423397 non-null  object 
 2   description     0 non-null       float64
 3   menus_appeared  423397 non-null  int64  
 4   times_appeared  423397 non-null  int64  
 5   first_appeared  423397 non-null  int64  
 6   last_appeared   423397 non-null  int64  
 7   lowest_price    394297 non-null  float64
 8   highest_price   394297 non-null  float64
dtypes: float64(3), int64(5), object(1)
memory usage: 29.1+ MB


In [10]:
menu_df.head()

,id,name,sponsor,event,venue,place,physical_description,occasion,notes,call_number,keywords,language,date,location,location_type,currency,currency_symbol,status,page_count,dish_count
0,12463,NaN,HOTEL EASTMAN,BREAKFAST,COMMERCIAL,"HOT SPRINGS, AR",CARD; 4.75X7.5;,EASTER;,NaN,1900-2822,NaN,NaN,1900-04-15,Hotel Eastman,NaN,NaN,NaN,complete,2,67
1,12464,NaN,REPUBLICAN HOUSE,[DINNER],COMMERCIAL,"MILWAUKEE, [WI];",CARD; ILLUS; COL; 7.0X9.0;,EASTER;,WEDGEWOOD BLUE CARD; WHITE EMBOSSED GREEK KEY ...,1900-2825,NaN,NaN,1900-04-15,Republican House,NaN,NaN,NaN,under review,2,34
2,12465,NaN,NORDDEUTSCHER LLOYD BREMEN,FRUHSTUCK/BREAKFAST;,COMMERCIAL,DAMPFER KAISER WILHELM DER GROSSE;,CARD; ILLU; COL; 5.5X8.0;,NaN,"MENU IN GERMAN AND ENGLISH; ILLUS, STEAMSHIP A...",1900-2827,NaN,NaN,1900-04-16,Norddeutscher Lloyd Bremen,NaN,NaN,NaN,complete,2,84
3,12466,NaN,NORDDEUTSCHER LLOYD BREMEN,LUNCH;,COMMERCIAL,DAMPFER KAISER WILHELM DER GROSSE;,CARD; ILLU; COL; 5.5X8.0;,NaN,"MENU IN GERMAN AND ENGLISH; ILLUS, HARBOR SCEN...",1900-2828,NaN,NaN,1900-04-16,Norddeutscher Lloyd Bremen,NaN,NaN,NaN,complete,2,63
4,12467,NaN,NORDDEUTSCHER LLOYD BREMEN,DINNER;,COMMERCIAL,DAMPFER KAISER WILHELM DER GROSSE;,FOLDER; ILLU; COL; 5.5X7.5;,NaN,"MENU IN GERMAN AND ENGLISH; ILLUS, HARBOR SCEN...",1900-2829,NaN,NaN,1900-04-16,Norddeutscher Lloyd Bremen,NaN,NaN,NaN,complete,4,33


In [11]:
menu_df.shape

(17545, 20)

In [12]:
menu_df.describe().round(2)

,id,keywords,language,location_type,page_count,dish_count
count,17545.00,0.0,0.0,0.0,17545.00,17545.00
mean,25325.95,NaN,NaN,NaN,3.48,75.62
std,6431.55,NaN,NaN,NaN,3.30,98.44
min,12463.00,NaN,NaN,NaN,1.00,0.00
25%,20742.00,NaN,NaN,NaN,2.00,20.00
50%,26165.00,NaN,NaN,NaN,2.00,35.00
75%,30707.00,NaN,NaN,NaN,4.00,93.00
max,35526.00,NaN,NaN,NaN,74.00,4053.00


In [13]:
menu_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17545 entries, 0 to 17544
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    17545 non-null  int64  
 1   name                  3197 non-null   object 
 2   sponsor               15984 non-null  object 
 3   event                 8154 non-null   object 
 4   venue                 8119 non-null   object 
 5   place                 8123 non-null   object 
 6   physical_description  14763 non-null  object 
 7   occasion              3791 non-null   object 
 8   notes                 10613 non-null  object 
 9   call_number           15983 non-null  object 
 10  keywords              0 non-null      float64
 11  language              0 non-null      float64
 12  date                  16959 non-null  object 
 13  location              17545 non-null  object 
 14  location_type         0 non-null      float64
 15  currency           

In [14]:
menuitem_df.head()

,id,menu_page_id,price,high_price,dish_id,created_at,updated_at,xpos,ypos
0,1,1389,0.4,NaN,1.0,2011-03-28 15:00:44 UTC,2011-04-19 04:33:15 UTC,0.111429,0.254735
1,2,1389,0.6,NaN,2.0,2011-03-28 15:01:13 UTC,2011-04-19 15:00:54 UTC,0.438571,0.254735
2,3,1389,0.4,NaN,3.0,2011-03-28 15:01:40 UTC,2011-04-19 19:10:05 UTC,0.140000,0.261922
3,4,1389,0.5,NaN,4.0,2011-03-28 15:01:51 UTC,2011-04-19 19:07:01 UTC,0.377143,0.262720
4,5,3079,0.5,1.0,5.0,2011-03-28 15:21:26 UTC,2011-04-13 15:25:27 UTC,0.105714,0.313178


In [15]:
menuitem_df.shape

(1332726, 9)

In [16]:
menuitem_df.describe().round(2)

,id,menu_page_id,price,high_price,dish_id,xpos,ypos
count,1332726.00,1332726.00,886810.00,91905.00,1332485.00,1332726.00,1332726.00
mean,697898.38,47594.87,12.84,8.11,158011.04,0.39,0.55
std,399980.67,22039.21,499.55,90.10,167762.04,0.22,0.22
min,1.00,130.00,0.00,0.00,1.00,0.00,0.00
25%,350251.25,32049.00,0.25,0.50,5089.00,0.18,0.37
50%,702410.50,53371.00,0.40,1.25,80700.00,0.38,0.57
75%,1045548.75,66823.00,1.00,3.00,332524.00,0.57,0.74
max,1385906.00,77425.00,180000.00,7800.00,515677.00,0.99,1.00


In [17]:
menuitem_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1332726 entries, 0 to 1332725
Data columns (total 9 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   id            1332726 non-null  int64  
 1   menu_page_id  1332726 non-null  int64  
 2   price         886810 non-null   float64
 3   high_price    91905 non-null    float64
 4   dish_id       1332485 non-null  float64
 5   created_at    1332726 non-null  object 
 6   updated_at    1332726 non-null  object 
 7   xpos          1332726 non-null  float64
 8   ypos          1332726 non-null  float64
dtypes: float64(5), int64(2), object(2)
memory usage: 91.5+ MB
